In [1]:
import csv
import glob
import pickle
import numpy as np
import pandas as pd
import networkx as nx
import dask.dataframe as dd
import multiprocessing as mp
import xml.etree.ElementTree as ET
#testing

* <s> How large a group of co-authors does the average publication have? </s>
* Is there a correlation between citations and the number of keywords that papers share? I.e. papers which share the same subject cite each other more often.
* For the most-cited papers (define your own cutoff), is the correlation in shared keywords between them and the papers that cite them different from (5) 

# Reading and combining pickles.

In [2]:
pickle_path_pa = '/students/2021-2022/master/Rients_DSLS/pickle_6/'
pickle_path_cite = '/students/2021-2022/master/Rients_DSLS/pickle_frames/'

pickles_pa = glob.glob(pickle_path_pa + 'pubmed21n*.pkl')
pickles_cite = glob.glob(pickle_path_cite + 'pubmed21n*.pkl')

#### Creating a dask dataframe out of all the given pickle files

In [4]:
# Creating a dataframe using dask
ddf2 = dd
ddf_list = []

# Own question requires pickles_cite, question1 requires pickles_pa
#for pickle in pickles_pa: 
for pickle in pickles_cite: 
    # Creating a pandas dataframe from each pickle
    df = pd.read_pickle(pickle)

    # Turning pandas df into a dask dataframe.
    ddf = dd.from_pandas(df, npartitions=8)
    ddf_list.append(ddf)   
    nr = len(ddf_list)
    print('list size: ', nr, '/1062')

print('Combining dataframes')
full_frame = dd.multi.concat(ddf_list)
# ~1 min for pickles paper author
# ~8 min for pickles citation graph

list size:  1 /1062
list size:  2 /1062
list size:  3 /1062
list size:  4 /1062
list size:  5 /1062
list size:  6 /1062
list size:  7 /1062
list size:  8 /1062
list size:  9 /1062
list size:  10 /1062
list size:  11 /1062
list size:  12 /1062
list size:  13 /1062
list size:  14 /1062
list size:  15 /1062
list size:  16 /1062
list size:  17 /1062
list size:  18 /1062
list size:  19 /1062
list size:  20 /1062
list size:  21 /1062
list size:  22 /1062
list size:  23 /1062
list size:  24 /1062
list size:  25 /1062
list size:  26 /1062
list size:  27 /1062
list size:  28 /1062
list size:  29 /1062
list size:  30 /1062
list size:  31 /1062
list size:  32 /1062
list size:  33 /1062
list size:  34 /1062
list size:  35 /1062
list size:  36 /1062
list size:  37 /1062
list size:  38 /1062
list size:  39 /1062
list size:  40 /1062
list size:  41 /1062
list size:  42 /1062
list size:  43 /1062
list size:  44 /1062
list size:  45 /1062
list size:  46 /1062
list size:  47 /1062
list size:  48 /1062
l

## Own question
What % of papers does not have a reference list?

In [121]:
unique_papers = full_frame['pubmedID'].nunique().compute()
unique_papers
# The total amount of papers is: 31847923

31847923

In [5]:
full_frame.head()

,pubmedID,Authors,Refs,keywords
0,5969596,"[G Bower, J McLean, J Meacham]",NaN,"[Animals, Birds, Conditioning, Psychological, ..."
1,5969597,"[R C Calfee, R C Atkinson]",NaN,"[Animals, Decision Making, Learning, Male, Pro..."
2,5969598,"[R C Bolles, L W Stokes, M S Younger]",NaN,"[Animals, Avoidance Learning, Conditioning, Ps..."
3,5969599,[F Nelson],NaN,"[Animals, Conditioning, Psychological, Electro..."
4,5969600,"[E Fantino, D Sharp, M Cole]",NaN,"[Animals, Avoidance Learning, Conditioning, Ps..."


In [6]:
full_frame_refs = full_frame[full_frame['Refs'].isna() == False]

In [7]:
full_frame_refs.head()

,pubmedID,Authors,Refs,keywords
75,5969679,[F Forster],"[13918530, 14255452]","[Adult, Demography, Epidemiology, Female, Huma..."
76,5969680,"[G R McNab, E J Slator, C J Stewart]","[13364352, 13700260, 14401755]","[Bronchitis, Chronic Disease, Cough, England, ..."
77,5969681,[J W Palmer],[14171095],"[Adolescent, Adult, Aged, Communication, Human..."
80,5969684,"[P Dukor, F M Dietrich, M Rosenthal]","[5320432, 5858699, 5932903, 5330737, 4958328, ...","[Animals, Antibody Formation, Female, Hemolysi..."
81,5969685,"[R K Main, L J Cole]","[14088079, 14107045, 14113075, 14162535, 14162...","[Animals, DNA, Female, In Vitro Techniques, Ma..."


In [123]:
len(full_frame_refs)
# The total amount of papers with references is: 6776743

6776743

In [124]:
6776743 / 31847923 * 100
# 21% of papers has references.

21.278445693303137

## Question 1
How large a group of co-authors does the average publication have?

How to answer: <br>
Check the total amount of authors divided by the amount of unique pubmedIDs.

In [128]:
# Question1.
Full_df = full_frame
total = len(Full_df) 
uniques = Full_df[0].nunique().compute()
average_authors = total / uniques

print('The papers have', average_authors, 'authors on average')

The papers have 4.1985290298819535 authors on average


## Question 5
Is there a correlation between citations and the number of keywords that papers share? I.e. papers which share the same subject cite each other more often.

So far only answered using 1 xml file instead of all of them.

In [34]:
cite_graph = nx.read_gpickle('/students/2021-2022/master/Rients_DSLS/pickle_graph/cite_small.pkl')
#cite_graph_full = nx.read_gpickle('/students/2021-2022/master/Rients_DSLS/pickle_graph/cite_graph.pkl')

In [35]:
# Loading all the subgraphs.
loaded_components = sorted(nx.connected_components(cite_graph), key=len, reverse=True)
# In total 727593 different subgraphs exist. 
total_cc = len(loaded_components)
total_cc

7117

In [36]:
# Making a subgraph of only the largest component.
cite_sg0 = cite_graph.subgraph(loaded_components[0])

In [72]:
# Checking the % of keywords that match between citations.

matches = []
no_matches = []
nodes_list = list(cite_sg0.nodes())
amount_nodes = len(nodes_list)



for n1, n2, k in list(cite_sg0.edges(data=True)):
    # Only check if they both have keywords.
    if cite_sg0.nodes[n1] != {} and cite_sg0.nodes[n2] != {}:
        #print(' node1: ', cite_sg0.nodes[n1]['keywords'],'\n', 'node2: ', cite_sg0.nodes[n2]['keywords'], '\n')
        key1 = cite_sg0.nodes[n1]['keywords']
        key2 = cite_sg0.nodes[n2]['keywords']


        # There are float nans, and i don't know how they got here.
        # 
        if key1 == key1 and key2 == key2:
            matches.append(len(set(key1).intersection(key2)) / max(len(set(key1)), len(set(key2))))

np.mean(matches)

0.25794364392706337

In [75]:
# Checking the % of keywords that match when there is no citation. 
# I calculate this by just taking two random papers and checking them for matching keywords.
# By random chance there could be citations included, however unless it happens a 100+ times, it will not have an effect.
# And while I could prevent this, I dont want to add a lot of processing time.


for n in range(0,50000): 
    n1 = nodes_list[np.random.randint(0,amount_nodes)]
    n2 = nodes_list[np.random.randint(0,amount_nodes)]

    # Incase both n1 and n2 end up being the same
    while n1 == n2:
        n2 = nodes_list[np.random.randint(0,amount_nodes)]
        print('What are the odds.')
        
    if cite_sg0.nodes[n1] != {} and cite_sg0.nodes[n2] != {}:
        n_key1 = cite_sg0.nodes[n1]['keywords']
        n_key2 = cite_sg0.nodes[n2]['keywords']

        if n_key1 == n_key1 and n_key2 == n_key2:
            no_matches.append(len(set(n_key1).intersection(n_key2)) / max(len(set(n_key1)),len(set(n_key2))))

In [76]:
cited_ans = int(np.mean(matches)*100)
non_cite_ans = int(np.mean(no_matches)*100)

print('average % of keywords shared from a cited paper: ',cited_ans)
print('average % of keywords shared between two random papers: ', non_cite_ans)

# 41 is significantly different than 6-8.

average % of keywords shared from a cited paper:  25
average % of keywords shared between two random papers:  5


# Writing the answers

In [134]:
# Writing the answers.
with open('answers.csv', mode='w') as csv_file:
    answer_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    answer_writer.writerow(['How large a group of co-authors does the average publication have?', round(average_authors,2)])
    answer_writer.writerow(['Is there a correlation between citations and the number of keywords that papers share?', \
    'Yes ' + str(cited_ans) + '% is significantly different than ' + str(non_cite_ans)+ '%'])

## Graph observations
for the author - paper graph

In [ ]:
loaded_graph = nx.read_gpickle('/students/2021-2022/master/Rients_DSLS/pickle_graph/full_graph.pkl')
# ~ 7 minutes.

In [ ]:
# In total there are 48996125 nodes.
total_nodes = len(loaded_graph)

In [ ]:
# Loading all the subgraphs.
loaded_components = sorted(nx.connected_components(loaded_graph), key=len, reverse=True)
# In total 727593 different subgraphs exist. 
total_cc = len(loaded_components)
total_cc
# ~ 3m 25 min

In [ ]:
# The first subgraph has 46720986 nodes
loaded_sg0 = loaded_graph.subgraph(loaded_components[0])
length_sg0 = len(loaded_sg0)
length_sg0

In [ ]:
# Creating another 4.5gb graph. 
# Weirdly enough 200mb larger than the complete graph.
# Using pickle is the most efficient.
#nx.write_gpickle(loaded_sg0, path='/students/2021-2022/master/Rients_DSLS/pickle_graph/full_sg.pkl')
# ~5 min

In [ ]:
# The second already drops down to 1101
# In this graph all articles are written by 'springer medizin' a german publication company. No co-authors and no connections.
loaded_sg1 = loaded_graph.subgraph(loaded_components[1])
length_sg1 = len(loaded_sg1)
length_sg1

In [ ]:
# 95% of all the nodes are in the first connected subgraph.
(length_sg0 / total_nodes)*100

I could try and incorperate all the subgraphs into my answers. <br>
But doing this will require a lot more work for I believe not a lot more answers. 

A lot of these graphs are old authors that have written a lot of papers with no co-author. <Br>
And also a lot of  'company authors' like 'Food and Drug Administration' and 'springer medizin' hiding the actual authors.

Therefore I will just take the first subgraph representing the 95% of data as main graph

In [ ]:
# This shows the nodes with the most connections. 
# Aka the authors with the most papers. (or the papers with the most authors but that won't show up in the top)
# It shows some authors with 15k + papers.
# Altough with very common names so it is probably more than 1 author but with the given xml file there is no way to check this.
sorted(loaded_sg0.degree, key=lambda x: x[1], reverse=True)[:10]
# ~ 6.5 min

[('Wei Wang', 15254),
 ('Wei Zhang', 13897),
 ('Wei Li', 11040),
 ('Jing Wang', 10499),
 ('Yan Li', 10418),
 ('Yan Wang', 9503),
 ('Jing Li', 9494),
 ('Lei Zhang', 9405),
 ('Yang Liu', 9296),
 ('Yan Zhang', 9282)]

In [ ]:
#Computing centrality
degCent = nx.degree_centrality(loaded_sg0)
#Descending order sorting centrality
degCent_sorted=dict(sorted(degCent.items(), key=lambda item: item[1],reverse=True))
# ~ 7 mins